In [1]:
#Training dataset

training_data = [
    ['Green', 3, 'Mango'],
    ['Yellow', 3, 'Mango'],
    ['Red', 3, 'Grape'],
    ['Red', 3, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

#Column Label
#Essas são as saídas da árvore

def unique_vale(rows,col):
    """Encontre os valores únicos de uma coluna em um dataset"""
    return set(row[col] for row in rows)

########
# Demo
# unique_vals (training_data, 0)
# unique_vals (training_data, 1)
########


def class_counts(rows):
    """Conta o número de cada tipo em um dataset"""
    
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

########
# Demo
# class_counts(training_data)
########

def is_numeric(value):
    """Testa se um valor é númérico"""
    return isinstance(value, int) or isinstance(value, float)


class Question:
    """Essa classe apenas guarda o 'número de uma coluna' (exemplo, 0 para a cor)"""
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        #Compara o valor guardado com um exemplo dado
        
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        #Esse método é só um auxiliar para impressão
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return f"Is {self.column} {condition} {str(self.value)}"
    

def partitions(rows, question):
    """
        Para cara linha do dataset, checa se combina com a questão. Se sim 
        adiciona a "true rows", se não a "false rows"
    """
    true_rows, false_rows = [], []
    
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
            
    return true_rows, false_rows

def gini(rows):
    """Calcula a impueza gini para cada linha de uma lista"""
    
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl]/float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

def info_gain(left, right, current_uncertaninty):
    """
        Calcula o ganho de informação entre um nó e seus nó filhos
    """
    
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertaninty - p * gini(left) - (1-p) * gini(right)

def find_best_split(rows):
    """
        Encontra a melhor questão iterando sobre cada fetaure/value
        calculando o ganho de informação
    """
    best_gain = 0
    best_question = None
    current_uncertaninty = gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features):
        
        values = set([row[col] for row in rows])
        
        for val in values:
            
            question = Question(col, val)
            
            true_rows, false_rows = partitions(rows, question)
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
                
            gain = info_gain(true_rows, false_rows, current_uncertaninty)
            
            if gain >= best_gain:
                best_gain, best_question = gain, question
    
    return best_gain, best_question


class Leaf:
    """
        
    """
    
    def __init__(self, rows):
        self.predictions = class_counts(rows)
        

class Decision_Node:
    """
    
    """
    
    def __init__(self,
                question,
                true_branch,
                false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        
def build_tree(rows):
    
    gain, question = find_best_split(rows)
    
    if gain == 0:
        return Leaf(rows)
    
    true_rows, false_rows = partitions(rows, question)
    
    true_branch = build_tree(true_rows)
    
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)

def print_tree(node, spacing=""):
    
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    
    print(spacing + str(node.question))
    
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    

def classify(row, node):
    
    if isinstance(node, Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
    
def print_leaf(counts):
    
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total *100)) + "%"
    return probs 

if __name__ == "__main__":
    
    my_tree = build_tree(training_data)
    
    print_tree(my_tree)
    
    testing_data =[
        ['Green', 3, 'Mango'],
        ['Yellow', 3, 'Mango'],
        ['Red', 3, 'Grape'],
        ['Red', 3, 'Grape'],
        ['Yellow', 3, 'Lemon'],
    ]
    
    for row in testing_data:
        print(f"Actual: {row[-1]}. Predicted: {print_leaf(classify(row, my_tree))}")
    

Is 0 == Red
--> True:
  Predict {'Grape': 2}
--> False:
  Is 0 == Yellow
  --> True:
    Predict {'Mango': 1, 'Lemon': 1}
  --> False:
    Predict {'Mango': 1}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Mango. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}
